# 1.准备工作

## 1.1导包

In [ ]:
from datetime import datetime
print(datetime.now())
#data preprocessing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os
import sys
import shutil
from collections import Counter, defaultdict
import torch
import gc
# from tqdm import tqdm

2025-05-07 05:06:59.955047


## 1.2设置一些常量

In [2]:
# the full input files pathes are here
DATA_PATH_stages="../data/kdigo_stages_measured.csv" 
DATA_PATH_labs = "../data/labs-kdigo_stages_measured.csv" 
DATA_PATH_vitals = "../data/vitals-kdigo_stages_measured.csv" 
DATA_PATH_vents = "../data/vents-vasopressor-sedatives-kdigo_stages_measured.csv"
DATA_PATH_detail="../data/icustay_detail-kdigo_stages_measured.csv" 
SEPARATOR=";"

# 输出路径
OUTPUT_PATH = "../data/AKI"

In [3]:
# 设置参数为常量

# 决定哪个分类任务将被执行, 每次只执行一个分类任务
ALL_STAGES = False # 如果为True，则执行多分类（0， 1， 2， 3）而不是二分类

CLASS1 = True     # 是否开启AKI的二分类，就是aki>=1都是1,否则为0
CLASS2 = False    # 是否开启中等AKI的二分类，就是aki>=2都是1,否则为0
CLASS3 = False    # 是否开启重度AKI的二分类，就是aki>=3都是1,否则为0

MAX_FEATURE_SET = True


# 数据重采样与缺失值处理
TIME_SAMPLING = True 
SAMPLING_INTERVAL = '1H'
# RESAMPLE_LIMIT = 16 # 4 days * 6h interval

# MOST_COMMON=False 表示：数值型变量用平均值（mean）填；类别型变量用最大值（出现最多的值）填。
MOST_COMMON = False 

# 模拟 ICU 预测任务的设定 —— 只使用前 48 小时的数据。
MAX_HOUR = 48

IMPUTE_EACH_ID = True           # 对每个 ICU stay 单独填补缺失值
IMPUTE_COLUMN = False           # 不用全列的统计信息填
IMPUTE_METHOD = 'most_frequent' 
FILL_VALUE = 0                  # 用于填充 3D数组中不规则位置（例如被Pad的地方）

# 年龄筛选（只用成年患者）
ADULTS_MIN_AGE = 18
ADULTS_MAX_AGE = -1

# 归一化与截断（异常值处理）
# 使用最常见的 min-max 归一化。
NORMALIZATION = 'min-max'
NORM_TYPE = 'min_max'

# 对特征做截断，把上/下 1% 的极端值剪掉，处理异常值。
CAPPING = True
if CAPPING:
    CAPPING_THRESHOLD_UPPER = 0.99
    CAPPING_THRESHOLD_LOWER = 0.01


# 使用随机划分训练/验证/测试集（而不是预设好的划分）
# 20% 数据用于验证或测试集。
# 随机种子是 42（保证可复现性）。
RANDOM_SPLIT = True
FIXED = False
RANDOM_SEED = 42
SPLIT_SIZE = 0.2 

# 特征列表定义

# min_set 是一个最小特征集，包含基础变量：id，时间点，肌酐，尿量，AKI阶段等。
min_set =  ["icustay_id", "charttime", "creat", "uo_rt_6hr", "uo_rt_12hr", "uo_rt_24hr", "aki_stage"]


# max_set 是一个扩展特征集，包含了更多实验指标、生理参数、性别、种族、入院类型等信息。
# 当前 MAX_FEATURE_SET = True，所以模型会使用 max_set
max_set = ['icustay_id', 'charttime', 'aki_stage', 'hadm_id', 'albumin_avg','aniongap_avg', 'bicarbonate_avg', 
           'bilirubin_avg', 'bun_avg','chloride_avg', 'creat', 'diasbp_mean', 'glucose_avg', 'heartrate_mean',
           'hematocrit_avg', 'hemoglobin_avg', 'potassium_avg', 'resprate_mean','sodium_avg', 'spo2_mean', 'sysbp_mean', 
           'uo_rt_12hr', 'uo_rt_24hr','uo_rt_6hr', 'wbc_avg', 'sedative', 'vasopressor', 'vent', 'age', 'F','M', 
           'asian', 'black', 'hispanic', 'native', 'other', 'unknown','white', 'ELECTIVE', 'EMERGENCY', 'URGENT']

## 1.3固定随机种子，保证结果可复现

In [4]:
# 固定随机种子, 保证可复现性
def same_seed(seed = 42): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
same_seed(RANDOM_SEED)

## 1.4设置一些全局函数

In [5]:
# Some functions used later

def cap_data(df):
    """
    对数值型特征进行异常值截断处理，将其限制在指定的分位数范围内。
    不处理的列包括 ID 和时间相关字段。
    """
    print(f"Capping between the {CAPPING_THRESHOLD_LOWER} and {CAPPING_THRESHOLD_UPPER} quantiles.")
    
    # 不参与截断的列
    exclude_columns = ['icustay_id', 'charttime', 'aki_stage', 'subject_id', 'intime', 'HOURS']
    # 筛选需要截断的列
    target_columns = df.columns.difference(exclude_columns)
    # 分位数截断
    lower_bound = df[target_columns].quantile(CAPPING_THRESHOLD_LOWER)
    upper_bound = df[target_columns].quantile(CAPPING_THRESHOLD_UPPER)
    
    df[target_columns] = df[target_columns].clip(lower=lower_bound, upper=upper_bound, axis=1)
    
    return df
 
    
def normalise_data(df, norm_mask):
    """
    对指定列使用 Min-Max 归一化，将值缩放至 [0, 1]。
    """
    print(f"Normalizing in [0,1] using {NORMALIZATION} normalization.")
    
    df[norm_mask] = (df[norm_mask] - df[norm_mask].min()) / (df[norm_mask].max() - df[norm_mask].min())
    
    return df


def fast_mode(df, key_cols, value_col):
    """
    对缺失值列按 key_cols 分组计算众数(不含 NaN),用于填充。
    若有多个众数，仅保留排序最靠前的。
    """
    grouped = df.groupby(key_cols + [value_col]).size().to_frame('counts').reset_index()
    top_values = grouped.sort_values('counts', ascending=False)
    mode_per_group = top_values.drop_duplicates(subset=key_cols).drop(columns='counts')
    return mode_per_group

## 2.读取csv文件

## 2.1 就是读csv

In [6]:
# 读取主数据集（包含AKI阶段、肌酐、尿量等）
print("读取 CSV 文件")
X = pd.read_csv(DATA_PATH_stages, sep=SEPARATOR)

# 删除冗余列，仅保留融合后的 aki_stage 标签
X.drop(["aki_stage_creat", "aki_stage_uo"], axis=1, inplace=True)

# 删除5个关键字段全部缺失的行（这些行无任何有效信息）
X = X.dropna(how='all', subset=['creat', 'uo_rt_6hr', 'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage'])

# 将 charttime 字段转换为 pandas 的 datetime 类型，便于后续时间处理
print("将 charttime 转换为时间戳")
X['charttime'] = pd.to_datetime(X['charttime'])

读取 CSV 文件
将 charttime 转换为时间戳


In [7]:
# 读取病人基本信息（包含 icustay_id、入院时间 intime、出生时间等）
dataset_detail = pd.read_csv(DATA_PATH_detail, sep=SEPARATOR)

# 删除与建模无关的静态信息，仅保留后续所需的字段（如 intime）
dataset_detail.drop([
    'dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity',
    'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay',
    'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay'
], axis=1, inplace=True)

# 将入ICU时间（intime）转换为 datetime 类型，便于计算时间差
print("将 intime 转换为时间戳")
dataset_detail['intime'] = pd.to_datetime(dataset_detail['intime'])

# dataset_detail的admission_age修改为age，原因我也不知道为什么MIT给的和我们参考的这个代码不一样
dataset_detail.rename(columns={'admission_age': 'age'}, inplace=True)

将 intime 转换为时间戳


In [8]:
# 构造 INTIME 表，用于后续将 ICU stay 的时间与数据记录做关联
INTIME = pd.DataFrame()
INTIME['icustay_id'] = dataset_detail['icustay_id']
INTIME['intime'] = dataset_detail['intime']

In [9]:
dataset_labs = pd.read_csv(DATA_PATH_labs, sep= SEPARATOR) # 'bands lactate platelet ptt inr pt
dataset_labs.drop(['albumin_min', 'albumin_max','bilirubin_min', 'bilirubin_max','bands_min', 'bands_max',
                   'lactate_min', 'lactate_max','platelet_min', 'platelet_max','ptt_min', 'ptt_max', 
                   'inr_min', 'inr_max', 'pt_min', 'pt_max'], axis = 1, inplace = True)
dataset_labs = dataset_labs.dropna(subset=['charttime'])
dataset_labs = dataset_labs.dropna(subset=dataset_labs.columns[4:], how='all')
dataset_labs['charttime'] = pd.to_datetime(dataset_labs['charttime'])
dataset_labs = dataset_labs.sort_values(by=['icustay_id', 'charttime'])

In [10]:
if  MAX_FEATURE_SET:
    dataset_vitals = pd.read_csv(DATA_PATH_vitals, sep= SEPARATOR)  
    
    dataset_vitals.drop(["heartrate_min", "heartrate_max","sysbp_min", "sysbp_max","diasbp_min", "diasbp_max",
                        'meanbp_min','meanbp_max', 'meanbp_mean','tempc_min', 'tempc_max', 'tempc_mean',
                        "resprate_min", "resprate_max", "spo2_min", "spo2_max", "glucose_min", "glucose_max"], axis = 1, inplace = True)    
    print("convert charttime to timestamp")
    dataset_vitals['charttime'] = pd.to_datetime(dataset_vitals['charttime'])
    dataset_vitals = dataset_vitals.sort_values(by=['icustay_id', 'charttime'])
    dataset_vitals = dataset_vitals.dropna(subset=dataset_vitals.columns[4:], how='all')

convert charttime to timestamp


In [11]:
if  MAX_FEATURE_SET:
    dataset_vents = pd.read_csv(DATA_PATH_vents , sep= SEPARATOR)
    print("convert charttime to timestamp")
    dataset_vents['charttime'] = pd.to_datetime(dataset_vents['charttime'])
    dataset_vents = dataset_vents.sort_values(by=['icustay_id', 'charttime'])

convert charttime to timestamp


## 2.1简单处理这几张csv表格，并将所有时间序列相关数据合成成一张大表

In [12]:
# labs的数据冗余大，且有些列是重复的（如 min/max），
# 需要对 labs 数据进行处理，计算 min/max 的平均值，并删除原有的 min/max 列
print("compute avg from min/max in labs file")
print(datetime.now())
# 并且从索引 4 开始的22列（共11对）为需要计算平均值的列，格式为：min, max, min, max, ...


df_info = dataset_labs.iloc[:, :4].copy() # 保存前4列数据不变
lab_cols = dataset_labs.columns[4:4+22] # 获取需要处理的22列
new_lab_cols = [] # 新建一个列表存放经过平均值处理后的新列数据
null_l = [] # 用来记录出现一侧为缺失而另一侧非缺失的行索引
changed = 0 # 用于累计更改的记录数

# 分组处理，每组两列，表示一项指标的min和max
for i in range(0, len(lab_cols), 2):
    col_min_name = lab_cols[i]
    col_max_name = lab_cols[i+1]
    
    col_min = dataset_labs[col_min_name]
    col_max = dataset_labs[col_max_name]
    
    # 计算平均值：(min + max)/2，注意如果其中一个是NaN，结果仍为NaN
    avg_val = (col_min + col_max) / 2
    
    # 判断条件：如果二者相等或均为NaN，则不更新；否则更新为平均值
    cond = ~((col_min == col_max) | (col_min.isna() & col_max.isna()))
    
    # 记录更改的个数
    changed_pair = cond.sum()
    changed += changed_pair
    
    # 若其中一边为NaN而另一边不为NaN，则记录该行索引
    null_condition = ((col_min.isna() & col_max.notna()) | (col_min.notna() & col_max.isna()))
    null_rows = dataset_labs.index[null_condition].tolist()
    null_l.extend(null_rows)
    
    # 创建新列，只有在 cond 条件下更新为平均值，否则保持原min值
    new_series = col_min.copy()
    new_series[cond] = avg_val[cond]
    # 添加到新列列表中
    new_lab_cols.append(new_series)

# 构造新的 DataFrame，将原来的前4列与新计算的11列合并
new_dataset_labs = pd.concat([df_info] + new_lab_cols, axis=1)
# 重新设置列名称（后面的顺序与论文中对应的实验指标名称相同）
new_dataset_labs.columns = ['subject_id', 'hadm_id', 'icustay_id', 'charttime',
                              'aniongap_avg', 'bicarbonate_avg', 'creatinine_avg', 'chloride_avg',
                              'glucose_avg', 'hematocrit_avg', 'hemoglobin_avg', 'potassium_avg',
                              'sodium_avg', 'bun_avg', 'wbc_avg']
dataset_labs = new_dataset_labs

if len(null_l) > 0:
    print("null values encountered")

print("Total records changed:", changed)
print(datetime.now())

compute avg from min/max in labs file
2025-05-07 04:14:25.486678
Total records changed: 4316
2025-05-07 04:14:26.154707


**这里是多数据源数据，考虑合并，为什么呢？**

以本研究中用于预测急性肾损伤（AKI）的关键变量——肌酐（Creatinine）为例，其在 
数据源中分别以 creatinine_avg 和 creat 的形式在不同视图中记
录。这两者分别来源于聚合后的实验室均值视图与原始实验室检验表，尽管其本
质上指向相同的生理指标，但在实际应用中存在显著差异。 

In [13]:
print("合并肌酐(creatinine)与葡萄糖(glucose)数据")
# ==== 肌酐处理 ====
# 提取实验室数据中的 creatinine（非缺失）
creat_l = dataset_labs[['icustay_id', 'charttime', 'creatinine_avg']].dropna(subset=['creatinine_avg'])
creat_l.rename(columns={"creatinine_avg": "creat"}, inplace=True)
# 提取标签数据 X 中的 creat（非缺失）
creat_x = X[['icustay_id', 'charttime', 'creat']].dropna(subset=['creat'])
# 合并两个来源的数据，并去重
creat_merged = pd.concat([creat_x, creat_l], ignore_index=True).drop_duplicates()
# 删除旧的 creat 字段
X.drop(columns=['creat'], inplace=True)
dataset_labs.drop(columns=['creatinine_avg'], inplace=True)
# 删除 labs 中除了 ID 和时间外全为空的行（表示该时间点无任何实验室记录）
dataset_labs.dropna(subset=dataset_labs.columns[4:], how='all', inplace=True)
# 合并新的 creat 回 X
X = pd.merge(X, creat_merged, on=["icustay_id", "charttime"], how="outer", sort=True, copy=False)



# ==== 葡萄糖处理 ====
if MAX_FEATURE_SET:
    # 提取 Vitals 中的 glucose（非缺失）
    glucose_v = dataset_vitals[
        ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'glucose_mean']
    ].dropna(subset=['glucose_mean'])
    glucose_v.rename(columns={"glucose_mean": "glucose_avg"}, inplace=True)
    # 提取 Labs 中的 glucose（非缺失）
    glucose_l = dataset_labs[
        ['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'glucose_avg']
    ].dropna(subset=['glucose_avg'])
    # 合并 vitals 与 labs 中的 glucose，并去重
    glucose_merged = pd.concat([glucose_l, glucose_v], ignore_index=True).drop_duplicates()
    # 删除原始 glucose 字段
    dataset_labs.drop(columns=['glucose_avg'], inplace=True)
    dataset_vitals.drop(columns=['glucose_mean'], inplace=True)
    # 删除 vitals 中除了 ID 和时间外全为空的行
    dataset_vitals.dropna(subset=dataset_vitals.columns[4:], how='all', inplace=True)
    # 合并 glucose_avg 到 dataset_labs
    dataset_labs = pd.merge(
        dataset_labs, glucose_merged,
        on=['subject_id', 'hadm_id', 'icustay_id', 'charttime'],
        how="outer", sort=True, copy=False
    )


# 最后，对两个主要数据集按 icustay_id + charttime 排序
X.sort_values(by=['icustay_id', 'charttime'], inplace=True, ignore_index=True)
dataset_labs.sort_values(by=['icustay_id', 'charttime'], inplace=True, ignore_index=True)

合并肌酐(creatinine)与葡萄糖(glucose)数据


In [14]:
print("Merging labs, vitals and vents files")
if MAX_FEATURE_SET:
    X = pd.merge(X, dataset_labs, on = ["icustay_id", "charttime"], how= "outer", copy = False)
    X = pd.merge(X, dataset_vitals, on = ["icustay_id", "charttime","subject_id", "hadm_id"], how= "outer", copy = False)
    X = pd.merge(X, dataset_vents, on = ["icustay_id", "charttime"], how= "outer", copy = False) 

Merging labs, vitals and vents files


In [15]:
# 回收dataset_labs，dataset_vitals，dataset_vents的内存
del dataset_labs, dataset_vitals, dataset_vents
gc.collect()

20

# 3.数据预处理

## 3.1 筛选年龄、住院时长、构造标签

首先，年龄必须成年，考虑的因素？

In [16]:
print("开始处理与时间相关的动态数据") 
print("即将过滤掉未成年患者。")
dataset_detail = dataset_detail.loc[dataset_detail['age'] >= ADULTS_MIN_AGE]
adults_icustay_id_list = dataset_detail['icustay_id'].unique()
X = X[X.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id'], ignore_index = True)
X = X.sort_values(by=['icustay_id', 'charttime'], ignore_index = True)
adults_icustay_id_list = np.sort(adults_icustay_id_list)

开始处理与时间相关的动态数据
即将过滤掉未成年患者。


其次，每条icustay_id所对应的记录时长必须大于48h

In [17]:
print("即将剔除 charttime 范围小于 48 小时的 ICU stay")
def more_than_HOURS_ahead(X):
    # 初始化列表用于记录不同类型的 ICU stay
    drop_list = []             # charttime 跨度 < 48 小时的 ICU stay
    los_list = []              # 所有保留 ICU stay 的住院时长（单位：天）
    long_stays_id = []         # 住院时间超过 35 天的 ICU stay
    last_charttime_list = []   # 与 long_stays_id 对应的最后 charttime

    # 计算每个 icustay_id 的首末 charttime，并求时间跨度
    stay_span_df = (
        X.groupby("icustay_id")["charttime"]
         .agg(['first', 'last'])  # 取每个 stay 的首末时间
         .reset_index()
    )

    # 计算 LOS（天）
    stay_span_df['los'] = (
        (stay_span_df['last'] - stay_span_df['first'])
        .dt.total_seconds() / 60 / 60 / 24  # 转换成天
    ).round(4)

    # 遍历每条 ICU stay 记录
    for _, row in stay_span_df.iterrows():
        icustay_id = row['icustay_id']
        los = row['los']

        if los < 48 / 24:  # 少于 48 小时
            drop_list.append(icustay_id)
        else:
            los_list.append(los)
            if los > 35:
                long_stays_id.append(icustay_id)
                last_charttime_list.append(row['last'])

    print(f"{len(long_stays_id)} long stays")
    print(f"这里有 {len(drop_list)} 条id对应的记录时间跨度小于48h")

    # 从 X 中删除 ICU stay < 48 小时的记录
    X = X[~X['icustay_id'].isin(drop_list)].copy()

    # 重新获取剩下的 ICU stay ID，并按时间排序
    id_list = X['icustay_id'].unique()
    X = X.sort_values(by=['icustay_id', 'charttime'], ignore_index=True)

    return id_list, X, long_stays_id, last_charttime_list

id_list, X, long_stays_id,last_charttime_list  = more_than_HOURS_ahead(X)

即将剔除 charttime 范围小于 48 小时的 ICU stay
2267 long stays
这里有 5506 条id对应的记录时间跨度小于48h


我们参考的论文只进行简单的二分类，并且aki阶段为1，2，3均设置为1，否则为0

In [18]:
print("binarise labels")
if ALL_STAGES:
    pass
elif CLASS1:
    X.loc[X['aki_stage'] > 1, 'aki_stage'] = 1
elif CLASS2:
    X.loc[X['aki_stage'] < 2, 'aki_stage'] = 0
    X.loc[X['aki_stage'] > 1, 'aki_stage'] = 1
elif CLASS3:
    X.loc[X['aki_stage'] < 3, 'aki_stage'] = 0
    X.loc[X['aki_stage'] > 2, 'aki_stage'] = 1

binarise labels


In [19]:
print("对于每个 ICU stay,只要在任意时刻出现过 AKI = 1,就将该 stay 的最终标签设为 1,否则为 0。")
def one_label_per_icustay(id_list, X):
    # 只筛选需要的列：icustay_id 和 aki_stage
    aki_df = X[['icustay_id', 'aki_stage']]

    # 按 icustay_id 分组，检查该组中是否有任何一个值等于 1
    label_df = (
        aki_df.groupby('icustay_id')['aki_stage']
              .apply(lambda x: (x == 1).any())
              .astype(int)  # True -> 1, False -> 0
              .reset_index()
              .rename(columns={'aki_stage': 'y_true'})
    )

    # 保证输出的 icustay_id 顺序与 id_list 保持一致
    label_df = label_df.set_index('icustay_id').reindex(id_list).reset_index()

    return label_df['y_true'].tolist()

target_list = one_label_per_icustay(id_list, X)

target = pd.DataFrame()
target['icustay_id'] = id_list
target['y_true'] = target_list


对于每个 ICU stay,只要在任意时刻出现过 AKI = 1,就将该 stay 的最终标签设为 1,否则为 0。


In [20]:
print("number of neg and pos label in target(whole stay)")
target['y_true'].value_counts()

number of neg and pos label in target(whole stay)


y_true
1    31284
0    15963
Name: count, dtype: int64

In [21]:
print("dataset drop AKI stages column")
X = X.drop(['aki_stage'], axis=1)

dataset drop AKI stages column


In [22]:
X = X.drop(['subject_id', 'hadm_id'], axis=1)

In [23]:
X.isnull().sum()

icustay_id                0
charttime                35
uo_rt_6hr           7913979
uo_rt_12hr          7913979
uo_rt_24hr          7913979
creat              10288441
aniongap_avg       10316556
bicarbonate_avg    10305236
chloride_avg       10239953
hematocrit_avg     10123189
hemoglobin_avg     10256761
potassium_avg      10037843
sodium_avg         10202743
bun_avg            10292666
wbc_avg            10349001
glucose_avg         9216948
heartrate_mean      4934900
sysbp_mean          5383127
diasbp_mean         5384801
resprate_mean       4899036
spo2_mean           5081490
vent                     35
vasopressor              35
sedative                 35
dtype: int64

## 3.2 重新采样数据


假设原始 ICU 数据如下（间隔不规律）：

| icustay\_id | charttime           | glucose | vasopressor |
| ----------- | ------------------- | ------- | ----------- |
| 100001      | 2020-01-01 01:27:00 | 150     | 0           |
| 100001      | 2020-01-01 01:55:00 | 130     | 1           |
| 100001      | 2020-01-01 03:10:00 | 140     | 0           |

重采样（按 1 小时）后将变为：

| icustay\_id | charttime           | glucose | vasopressor |
| ----------- | ------------------- | ------- | ----------- |
| 100001      | 2020-01-01 01:00:00 | 140     | 1           |
| 100001      | 2020-01-01 02:00:00 | NaN     | NaN         |
| 100001      | 2020-01-01 03:00:00 | 140     | 0           |



In [24]:
print(f"开始时间重采样：每 {SAMPLING_INTERVAL} 一次")

# 设置不参与聚合的列
skip = ['icustay_id', 'charttime']
if MAX_FEATURE_SET:
    discrete_feat = ['sedative', 'vasopressor', 'vent']
    skip.extend(discrete_feat)
else:
    discrete_feat = []

# 推断数值特征（排除 ID、时间和离散特征）
numeric_feat = list(X.columns.difference(skip))

# 设定重采样方法
X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)

# 分别处理 numeric 和 discrete 特征
resampled_parts = []

if numeric_feat:
    X_numeric = X[numeric_feat].mean()
    resampled_parts.append(X_numeric)

if discrete_feat:
    X_discrete = X[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
    resampled_parts.append(X_discrete)

# 合并数值与离散部分（如果有）
X = pd.concat(resampled_parts, axis=1).reset_index()

print(f"重采样完成，当前数据形状：{X.shape}")


开始时间重采样：每 1H 一次


C:\Users\Doog\AppData\Local\Temp\ipykernel_3124\1014288339.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


重采样完成，当前数据形状：(13365741, 24)


### 按 Yereva 项目的时间窗口设定，只保留 ICU 前 48 小时内的数据

In [35]:
print("正在将 ICU 入院时间(intime)合并到主数据 X 中")
print("正在删除 ICU 内时间超出 48 小时或早于入院时间的记录(HOURS < 0 或 > 48)")
X = pd.merge(X, INTIME, on=["icustay_id"], how="left", copy=False)# 合并入 ICU 时间
X['HOURS'] = (X.charttime - X.intime).apply(lambda s: s / np.timedelta64(1, 's')) / 3600 # 计算每条记录距离入 ICU 的时间（单位：小时）
X = X[X['HOURS'] >= 0] # 删除 ICU 入住前的数据（负时间）
X = X[X['HOURS'] <= MAX_HOUR] # 删除超出 48 小时范围的数据（只保留前 MAX_HOUR 小时）
X = X.reset_index(drop=True)

正在将 ICU 入院时间(intime)合并到主数据 X 中
正在删除 ICU 内时间超出 48 小时或早于入院时间的记录(HOURS < 0 或 > 48)


In [36]:
X.shape

(2246888, 26)

In [39]:
#check if follows one hour interval
# X.loc[X["icustay_id"]== 272725].sort_values(by=['HOURS'])["HOURS"]
# X.loc[X["icustay_id"]== 244882].sort_values(by=['HOURS'])["HOURS"]
X.loc[X["icustay_id"]== 217128].sort_values(by=['HOURS'])["HOURS"]

386827     0.22
386828     1.22
386829     2.22
386830     3.22
386831     4.22
386832     5.22
386833     6.22
386834     7.22
386835     8.22
386836     9.22
386837    10.22
386838    11.22
386839    12.22
386840    13.22
386841    14.22
386842    15.22
386843    16.22
386844    17.22
386845    18.22
386846    19.22
386847    20.22
386848    21.22
386849    22.22
386850    23.22
386851    24.22
386852    25.22
386853    26.22
386854    27.22
386855    28.22
386856    29.22
386857    30.22
386858    31.22
386859    32.22
386860    33.22
386861    34.22
386862    35.22
386863    36.22
386864    37.22
386865    38.22
386866    39.22
386867    40.22
386868    41.22
386869    42.22
386870    43.22
386871    44.22
386872    45.22
386873    46.22
386874    47.22
Name: HOURS, dtype: float64

## 3.3 插值（缺失值填充）

In [41]:
print("Imputation.")
remove_list = ['icustay_id','charttime','intime','HOURS']

# 对 ICU 数据中的缺失值（NaN）进行按 ICU stay（icustay_id）内众数填充（most common value
if IMPUTE_EACH_ID:
    column_name = list(X.columns)
    for item in remove_list:
        column_name.remove(item)
    for feature in column_name:
        X.loc[X[feature].isnull(), feature] = X.icustay_id.map(fast_mode(X, ['icustay_id'], feature).set_index('icustay_id')[feature])       

# 以下代码不走我就注释了
# # imputation based on whole column
# if IMPUTE_COLUMN:
#     imp = SimpleImputer(missing_values=np.nan, strategy= IMPUTE_METHOD)
#     cols = list(X.columns)
#     for item in remove_list:
#         cols.remove(item)
#     X[cols]=imp.fit_transform(X[cols])  

# 对剩余仍为 NaN 的值填充为 FILL_VALUE（兜底
X = X.fillna(FILL_VALUE)

Imputation.


In [44]:
# more comfortable to review in this order
print("check variables")
try:
    cols = ['icustay_id', 'charttime','aniongap_avg','bicarbonate_avg', 'bun_avg','chloride_avg',
            'creat','diasbp_mean', 'glucose_avg', 'heartrate_mean', 'hematocrit_avg','hemoglobin_avg', 
            'potassium_avg', 'resprate_mean', 'sodium_avg','spo2_mean', 'sysbp_mean', 'uo_rt_12hr', 
            'uo_rt_24hr', 'uo_rt_6hr','wbc_avg', 'sedative', 'vasopressor', 'vent',"HOURS" , "intime"]
    X = X[cols]
    print("success")
except:
    try:
        cols = ['icustay_id', 'charttime','creat','uo_rt_12hr', 'uo_rt_24hr', 'uo_rt_6hr']
        X = X[cols]
    except:
        print("error")

check variables
success


In [45]:
X.shape

(2246888, 26)

## 3.4 构造类别特征（独热编码）

In [46]:
if MAX_FEATURE_SET:
    #extract datasets based on id_list
    dataset_detail = dataset_detail.loc[dataset_detail['icustay_id'].isin(id_list)]
    #sort by ascending order
    dataset_detail = dataset_detail.sort_values(by=['icustay_id'])
    #print(dataset_detail)
    
    #transfrom categorical data to binary form
    dataset_detail = dataset_detail.drop(['intime'], axis=1)
    dataset_detail = dataset_detail.join(pd.get_dummies(dataset_detail.pop('gender')))
    dataset_detail = dataset_detail.join(pd.get_dummies(dataset_detail.pop("ethnicity_grouped")))
    dataset_detail = dataset_detail.join(pd.get_dummies(dataset_detail.pop('admission_type')))
    #X = X.drop(['subject_id', 'hadm_id'], axis=1)
    #dataset_detail = dataset_detail.drop(['subject_id', 'hadm_id'], axis=1)
    X =  pd.merge(X, dataset_detail, on = ["icustay_id"], how= "left", copy = False) 

    numeric_feat.append('age')

In [49]:
X = X.drop(['charttime', 'intime'], axis=1)

In [ ]:
# 将X的后12个变量改为float32类型
X.iloc[:, -12:] = X.iloc[:, -12:].astype(np.float32)

In [52]:
feature_names =['Anion gap', 'Bicarbonate', 'Blood Urea Nitrogen', 'Chloride', 'Creatinine', 'Diastolic BP', 'Glucose', 'Heart rate', 
            'Hematocrit', 'Hemoglobin', 'Potassium', 'Respiratory rate', 'Sodium', 'Oxygen saturation', 'Systolic BP', 'Urine output 12h', 'Urine output 24h', 'Urine output 6h',
            'White cell count', 'Sedative', 'Vasopressor', 'Ventilation', 'Age', 'Female gender', 'Male gender', 'Asian ethnicity', 'Black ethnicity', 'Hispanic ethnicity', 'Native american', 
            'Other ethnicity', 'Ethnicity unknown', 'White ethnicity', 'Elective admission', 'Emergency admission', 'Urgent admission']

## 3.5 剔除极端值，标准化

In [55]:
X = cap_data(X)

X = normalise_data(X, numeric_feat)

Capping between the 0.01 and 0.99 quantiles.
Normalizing in [0,1] using min-max normalization.


In [63]:

seq_lengths = X.groupby(['icustay_id'],as_index=False).size().sort_values(by = ['size'],ascending=False)
sequence_length = seq_lengths.max() # the longest sequence per icustay-id
print(sequence_length)

icustay_id    299999
size              49
dtype: int64


In [64]:
#AL re-write as try except to make it work as hadm_id is not used if only one csv file is used and none are merged
try:
    X.drop(['hadm_id'], axis=1, inplace = True)
except:
    pass

In [65]:
X = X.sort_values(by=['subject_id', 'HOURS'])

### 算算最终参与数据集构造的数据特征量多少

In [68]:
features_list = list(X.columns)
print(features_list)
# list of variables to be removed at the end
remove_list_final = ['icustay_id', 'subject_id', 'F']
for item in remove_list_final:
    features_list.remove(item)

features = len(features_list)
print("number of features: " + str(features))

['icustay_id', 'aniongap_avg', 'bicarbonate_avg', 'bun_avg', 'chloride_avg', 'creat', 'diasbp_mean', 'glucose_avg', 'heartrate_mean', 'hematocrit_avg', 'hemoglobin_avg', 'potassium_avg', 'resprate_mean', 'sodium_avg', 'spo2_mean', 'sysbp_mean', 'uo_rt_12hr', 'uo_rt_24hr', 'uo_rt_6hr', 'wbc_avg', 'sedative', 'vasopressor', 'vent', 'HOURS', 'subject_id', 'age', 'F', 'M', 'asian', 'black', 'hispanic', 'native', 'other', 'unknown', 'white', 'ELECTIVE', 'EMERGENCY', 'URGENT']
number of features: 35


# 4. 按照subject_id随机划分训练集，验证集，测试集。

In [69]:
if RANDOM_SPLIT:
    print("extract subject_id list")
    subject_id = X["subject_id"].unique()
    subject_id = np.sort(subject_id)

extract subject_id list


In [70]:
if RANDOM_SPLIT:
    print("number of unique subject id: " + str(len(subject_id)))

number of unique subject id: 34280


In [71]:
if RANDOM_SPLIT:
    print('RANDOM SPLIT')
    print("divide dataset into train, test and validation sets")
    id_train_val, id_test = train_test_split(subject_id, test_size = 0.1, random_state = RANDOM_SEED) # train set is 80%)
    print("test is %d" % len(id_test))
    # remaining 20% split in halves as test and validation 10% and 10%
    id_train, id_val = train_test_split(id_train_val, test_size = 0.111, random_state = RANDOM_SEED) # test 10% valid 10%
    print("train is %d" %len(id_train))
    print("val is %d" %len(id_val))

    #sort list
    id_test.sort()
    id_train.sort()
    id_val.sort()

RANDOM SPLIT
divide dataset into train, test and validation sets
test is 3428
train is 27427
val is 3425


# 5. 将 icustay 数据转换为单独的时间序列 CSV 文件。

In [ ]:
#str(list(target.loc[target['icustay_id'] == 237693]['y_true'])[0])

In [ ]:
def convert_icustay_to_AKIfolder(dataset, subject_id, output_path, id_train, id_test, id_val, target):
    
    temp_icustay_list = [] #to store the icustay_id under same subject_id
    n = 0 #index to loop through temp_icustay_list
    num_stay = 0
    temp_dataset = pd.DataFrame()
    sub_temp_dataset = pd.DataFrame()
    
    train_pairs = []
    test_pairs = []
    val_pairs = []
    

    for subject in subject_id:
        #make path for subject folder
        dn = os.path.join(output_path, str(subject))
        try:
            os.makedirs(dn)
        except:
            pass
        
        temp_dataset = dataset.loc[dataset["subject_id"]== subject].sort_values(by=['icustay_id'])
        temp_icustay_list = temp_dataset["icustay_id"].unique()
        num_stay = len(temp_icustay_list)
        n = 0
        
        while n < num_stay:
            sub_temp_dataset = temp_dataset.loc[temp_dataset["icustay_id"]== temp_icustay_list[n]]
            sub_temp_dataset = sub_temp_dataset.drop(remove_list_final, axis=1)
            sub_temp_dataset = sub_temp_dataset.set_index('HOURS').sort_index(axis=0)
             
            sub_temp_dataset.to_csv(os.path.join(OUTPUT_PATH, str(subject), '{}_episode{}_timeseries_{}.csv'.format(subject, n+1, temp_icustay_list[n]))
                                    ,index_label='Hours')
            
            #create list for id list for train/test/val
            if subject in id_train:
                train_pairs.append((str(subject)+ "_note.txt", str(subject)+ "_episode" + str(n+1)+"_timeseries_"+ str(temp_icustay_list[n])+ ".csv", str(list(target.loc[target['icustay_id'] == temp_icustay_list[n]]['y_true'])[0])))
            elif subject in id_test:
                test_pairs.append((str(subject)+ "_note.txt", str(subject)+ "_episode" + str(n+1)+"_timeseries_"+ str(temp_icustay_list[n])+ ".csv", str(list(target.loc[target['icustay_id'] == temp_icustay_list[n]]['y_true'])[0])))
            elif subject in id_val:
                val_pairs.append((str(subject)+ "_note.txt", str(subject)+ "_episode" + str(n+1)+"_timeseries_"+ str(temp_icustay_list[n])+ ".csv", str(list(target.loc[target['icustay_id'] == temp_icustay_list[n]]['y_true'])[0])))
                
            n = n+1
    
    return train_pairs, test_pairs, val_pairs
    
    
train_pairs, test_pairs, val_pairs = convert_icustay_to_AKIfolder(X, subject_id, OUTPUT_PATH, id_train, id_test, id_val, target)        

# 移动 subject_timeseries.csv 文件 到 train/test 目录中

In [80]:
def move_to_partition(subjects_root_path, patients, partition):
    if not os.path.exists(os.path.join(subjects_root_path, partition)):
        os.mkdir(os.path.join(subjects_root_path, partition))
    for patient in patients:
        src = os.path.join(subjects_root_path, str(patient))
        dest = os.path.join(subjects_root_path, partition)
        for filename in os.listdir(src):
            shutil.move(os.path.join(src, str(filename)), dest)
        os.rmdir(src)

In [81]:
move_to_partition(OUTPUT_PATH, id_train, "train")
move_to_partition(OUTPUT_PATH, id_val, "train")
move_to_partition(OUTPUT_PATH, id_test, "test")

# 创建 test_listfile.csv、train_listfile.csv、val_listfile.csv，并移动到 AKI 文件夹中。

In [82]:
with open(os.path.join(OUTPUT_PATH, "train_listfile.csv"), "w") as listfile:
    listfile.write('notes,stay,y_true\n')
    for (n, x, y) in train_pairs:
        listfile.write('{},{},{}\n'.format(n, x, str(y)))
with open(os.path.join(OUTPUT_PATH, "val_listfile.csv"), "w") as listfile:
    listfile.write('notes,stay,y_true\n')
    for (n, x, y) in val_pairs:
        listfile.write('{},{},{}\n'.format(n, x, str(y))) 
        
with open(os.path.join(OUTPUT_PATH,  "test_listfile.csv"), "w") as listfile:
    listfile.write('notes,stay,y_true\n')
    for (n, x, y) in test_pairs:
        listfile.write('{},{},{}\n'.format(n, x, str(y))) 